In [ ]:
"""
Description:
    用于demo一路sEMG&FMG信号进行肌力识别
"""
import pandas as pd

from motiongui import *
from iFEMGprocessing import read_label, pdtable_read_db

In [ ]:
# read data
db_path = ""
label_path = ""
raw_data = pd.read_table(db_path, sep = ';', header = None)
label = read_label(label_path)



# 读取数据array
    data_time = raw_data[0].values
    data_FMG = raw_data[7].values
    raw_sEMG = raw_data[15].values
    
    # 滤波
    sEMGf1 = band_pass_filter(raw_sEMG, 1200, 15, 500)
    sEMGf2 = band_trap_filter(sEMGf1, 1200, 200)
    data_sEMG = band_trap_filter(sEMGf2, 1200, 400)
    
    #将db文件中的时间转换为ms level时间戳
    t_stamp = [] # 保存数据文件中时间转换的时间戳，精度ms
    for t in data_time:
        t_array = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S,%f")
        ret_stamp = int(time.mktime(t_array.timetuple()) * 1000 + t_array.microsecond/1000)
        t_stamp.append(ret_stamp)
        
    #处理label.txt中的ms level时间戳
    # label = read_label("D:\code\data\iFEMG\g-0.txt")
    label_t_stamp = [] # 保存label文件中时间对应的时间戳，精度ms
    for x in label:
        t = x[0] + " " + x[1]
        t_array = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S.%f")
        ret_stamp = int(time.mktime(t_array.timetuple()) * 1000 + t_array.microsecond/1000)
        label_t_stamp.append(ret_stamp)
        
    # 比较数据文件和label文件中时间戳，划分肌电活动段
    # 储存分割好的数据段
    sEMG_data_set = []
    FMG_data_set = []
    rsEMG_data_set = []
    rFMG_data_set = []
    # 临时一段静息数据和激活数据
    temp_sEMG = []
    temp_FMG = []
    temp_sEMG_r = [] # 静息数据
    temp_FMG_r = []

    for i in range(len(label) - 1): # 在label时间范围内
        if (label[i][2] == "收缩") and (label[i + 1][2] == "舒张"): # 活动段
            for j in range(len(t_stamp)): # 在整个data长度内搜索，可以优化
                if label_t_stamp[i] <= t_stamp[j] <= label_t_stamp[i + 1]:
                    temp_sEMG.append(data_sEMG[j])
                    temp_FMG.append(data_FMG[j])
            if len(temp_FMG) != 0:
                sEMG_data_set.append(temp_sEMG)
                FMG_data_set.append(temp_FMG)
            temp_sEMG = []
            temp_FMG = []
        else: # 非活动段，肌肉静息
            for j in range(len(t_stamp)):
                if label_t_stamp[i] <= t_stamp[j] <= label_t_stamp[i + 1]:
                    temp_sEMG_r.append(data_sEMG[j])
                    temp_FMG_r.append(data_FMG[j])
            rsEMG_data_set.append(temp_sEMG)
            rFMG_data_set.append(temp_FMG)
            temp_sEMG_r = []
            temp_FMG_r = []






sEMG, FMG, rsEMG, rFMG = data_segment(raw_data, label)
df = pd.DataFrame(columns = ('subject', 'strength_level', 'mf', 'mpf', 'power', 'power_time', 'FMG_mean'))

data_set_num = min([len(FMG), len(rFMG)])
for i in range(data_set_num):
    mf, mpf, power, power_time = sEMG_analysis(sEMG[i], 1200)
    FMG_mean = FMG_analysis(FMG[i], rFMG[i], 1200)
    df = df.append({'subject': subject,
                    'strength_level': strength_level,
                    'mf': mf,
                    'mpf': mpf,
                    'power': power,
                    'power_time': power_time,
                    'FMG_mean': FMG_mean}, ignore_index=True)